In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [ ]:
path_df = r"Table" + "\\"
path_model = r"Model" + "\\"
path_fig = r"Figure" + "\\"

file_data = path_df + "Data2_without_nan.csv"

N1 = 60  # Number of initial features
N2 = 30  # Number of final features

In [ ]:
df = pd.read_csv(file_data)
# df = df.sample(frac=0.05, random_state=10)

X = df.drop(columns=["River", "Station", "Date", "Discharge"])
y = df["Discharge"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
def calc_metric(y_true, y_pred, decimal_digit=3, name=None):
    from sklearn.metrics import r2_score, mean_absolute_error

    R = np.corrcoef(y_true, y_pred)[0, 1]
    R2 = r2_score(y_true, y_pred)
    MAE = mean_absolute_error(y_true, y_pred)
    MAPE = (
        np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    )  # Mean Absolute Percentage Error
    try:
        from sklearn.metrics import mean_squared_error

        RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
    except ImportError:
        from sklearn.metrics import root_mean_squared_error

        RMSE = root_mean_squared_error(y_true, y_pred)

    RRMSE = (RMSE / np.mean(y_true)) * 100
    fRMSE = RMSE / np.std(y_true)
    NRMSE = RMSE / (np.max(y_true) - np.min(y_true))
    PBIAS = (np.sum(y_pred - y_true) / np.sum(y_true)) * 100
    DDR = np.mean(
        np.abs((y_pred - y_true) / np.mean(y_true))
    )  # Developed Discrepancy Ratio (custom calculation)
    IQR = np.percentile(y_true, 75) - np.percentile(y_true, 25)  # Inter-Quartile Range
    RPIQ = IQR / RMSE  # Ratio of Performance to Inter-Quartile Range

    # Calculate Kling-Gupta Efficiency
    alpha = np.std(y_pred) / np.std(y_true)
    beta = np.mean(y_pred) / np.mean(y_true)
    KGE = 1 - np.sqrt((R - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

    # Store metrics
    metrics = {
        "R": R,
        "R2": R2,
        "MAE": MAE,
        "MAPE (%)": MAPE,
        "RMSE": RMSE,
        "RRMSE (%)": RRMSE,
        "fRMSE": fRMSE,
        "NRMSE": NRMSE,
        "PBIAS (%)": PBIAS,
        "DDR": DDR,
        "RPIQ": RPIQ,
        "KGE": KGE,
    }
    metrics = pd.Series(metrics).round(decimal_digit)
    if name:
        metrics.name = name

    return metrics


def get_best_model(metric):
    N = metric.shape[0]

    # Extract the score data for comparison
    scores = metric.iloc[:, 3:].copy()

    # Define metrics where higher is better and where lower is better
    list1 = ["R", "R2", "DDR", "RPIQ", "KGE"]  # Higher is better
    list2 = [
        "MAE",
        "RMSE",
        "RRMSE (%)",
        "fRMSE",
        "NRMSE",
        "PBIAS (%)",
    ]  # Lower is better

    # Initialize an empty DataFrame to store scores with points
    scores_with_points = pd.DataFrame(index=scores.index)

    # Use absolute values for PBIAS (%) for comparison
    if "PBIAS (%)" in scores.columns:
        scores["PBIAS (%)"] = scores["PBIAS (%)"].abs()

    # Define a function to handle inf values
    def apply_inf_handling(x, N, ascending):
        # Check if there are any inf values
        if np.isinf(x).any():
            return pd.Series([0] * N)  # Assign zero points to all records
        else:
            # Calculate normal rankings
            return x.rank(ascending=ascending, method="min").apply(lambda r: N - r + 1)

    # Calculate ranking scores for metrics where higher is better
    if list1:
        ranked_high_scores = scores[list1].apply(
            lambda x: apply_inf_handling(x, N, ascending=False), axis=0
        )
        scores_with_points[list1] = ranked_high_scores

    # Calculate ranking scores for metrics where lower is better
    if list2:
        ranked_low_scores = scores[list2].apply(
            lambda x: apply_inf_handling(x, N, ascending=True), axis=0
        )
        scores_with_points[list2] = ranked_low_scores

    # Calculate total score
    metric["Total Score"] = scores_with_points.sum(axis=1)

    # Find the record with the highest total score
    idx_max = metric["Total Score"].idxmax()
    best_model_row = metric.iloc[int(idx_max), :]

    print("The best model is:", best_model_row["Model"])

    # Save the information of the best model to a CSV file
    metric.loc[idx_max, "Best Model"] = 1
    metric.to_csv(path_df + "Model_performance.csv", index=False)

# Random Forest

## Feature Selection

In [ ]:
# Initial feature selection by feature importance ranking
model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

feature_importances = model.feature_importances_
feature_initial = np.argsort(feature_importances)[::-1][:N1]

X_train1 = X_train.iloc[:, feature_initial]
X_test1 = X_test.iloc[:, feature_initial]

In [ ]:
# Final feature selection by RFE
rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rfe = RFE(estimator=rf, n_features_to_select=N2, step=1)
rfe.fit(X_train1, y_train)

ranking = rfe.ranking_
ranking = pd.DataFrame(ranking, index=X_train1.columns, columns=["Rank"]).sort_values(
    by="Rank", ascending=True
)
ranking.index.name = "Feature"

features_selected = ranking[ranking["Rank"] == 1]
features_selected.to_csv(path_df + "Selected_features_RF.csv")

X_train2 = X_train1[features_selected.index]
X_test2 = X_test1[features_selected.index]

## Hyperparameter Optimization

In [ ]:
# Define the hyperparameter grid
param_grid = {
    "max_depth": [10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "n_estimators": [50, 100, 150],
}

# Random search of parameters
rf = RandomForestRegressor(random_state=42, bootstrap=True, n_jobs=-1)
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=120,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)
rf_random.fit(X_train2, y_train)

# Get the best model
model = rf_random.best_estimator_
best_params = rf_random.best_params_

# Save the model parameters
with open(path_model + "RF_best_params.json", "w") as f:
    json.dump(best_params, f)

## Performance Validation

In [ ]:
y_pred = model.predict(X_test2)
df_pred = pd.DataFrame({"Observed": y_test, "Predicted": y_pred})
df_pred.to_csv(path_df + "RF_Prediction.csv")

m1 = calc_metric(y_test, y_pred, 3, "RF")

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.scatter(y_test, y_pred, s=2)

min_value = min(min(y_test), min(y_pred))
max_value = max(max(y_test), max(y_pred))
ax.plot(
    [min_value, max_value],
    [min_value, max_value],
    color="grey",
    linewidth=1,
    linestyle="--",
)

ax.set_xlabel("In-situ Discharge (m³/s)")
ax.set_ylabel("RF Predicted Discharge (m³/s)")

fig.tight_layout()
plt.savefig(path_fig + "Scatter_RF.png", dpi=300)

# XGBoost

## Feature Selection

In [ ]:
# Initial feature selection by feature importance ranking
model = XGBRegressor(n_estimators=50, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

feature_importances = model.feature_importances_
feature_initial = np.argsort(feature_importances)[::-1][:N1]

X_train1 = X_train.iloc[:, feature_initial]
X_test1 = X_test.iloc[:, feature_initial]

In [ ]:
# Final feature selection by RFE
xgb = XGBRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rfe = RFE(estimator=xgb, n_features_to_select=N2, step=1)
rfe.fit(X_train1, y_train)

ranking = rfe.ranking_
ranking = pd.DataFrame(ranking, index=X_train1.columns, columns=["Rank"]).sort_values(
    by="Rank", ascending=True
)
ranking.index.name = "Feature"

features_selected = ranking[ranking["Rank"] == 1]
features_selected.to_csv(path_df + "Selected_features_XGB.csv")

X_train2 = X_train1[features_selected.index]
X_test2 = X_test1[features_selected.index]

## Hyperparameter Optimization

In [ ]:
# Define the hyperparameter grid
param_grid = {
    "max_depth": [3, 4, 5],
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [50, 100, 150],
    "subsample": [0.5, 0.7, 1],
    "colsample_bytree": [0.5, 0.7, 1],
}

# Random search of parameters
xgb = XGBRegressor(random_state=42, n_jobs=-1)
xgb_random = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)
xgb_random.fit(X_train2, y_train)

# Get the best model
model = xgb_random.best_estimator_
best_params = xgb_random.best_params_

# Save the model parameters
with open(path_model + "XGB_best_params.json", "w") as f:
    json.dump(best_params, f)

## Performance Validation

In [ ]:
y_pred = model.predict(X_test2)
df_pred = pd.DataFrame({"Observed": y_test, "Predicted": y_pred})
df_pred.to_csv(path_df + "XGB_Prediction.csv")

m2 = calc_metric(y_test, y_pred, 3, "XGB")

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.scatter(y_test, y_pred, s=2)

min_value = min(min(y_test), min(y_pred))
max_value = max(max(y_test), max(y_pred))
ax.plot(
    [min_value, max_value],
    [min_value, max_value],
    color="grey",
    linewidth=1,
    linestyle="--",
)

ax.set_xlabel("In-situ Discharge (m³/s)")
ax.set_ylabel("XGB Predicted Discharge (m³/s)")

fig.tight_layout()
plt.savefig(path_fig + "Scatter_XGB.png", dpi=300)

# Summary

In [60]:
metrics = pd.concat([m1, m2], axis=1).T
metrics.index.name = "Model"
metrics.reset_index(drop=False, inplace=True)

In [ ]:
river = df["River"].iloc[0]
station = df["Station"].iloc[0]

metrics.insert(0, "River", river)
metrics.insert(1, "Station", station)

metrics

In [ ]:
get_best_model(metrics)